<a href="https://colab.research.google.com/github/doronschwartz/NLP/blob/main/HW2/WeatherClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load in the Data

In [6]:
from google.colab import drive
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


Mount Google Drive and load the DataSet of pictures

In [7]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the dataset folder
dataset_path = '/content/drive/My Drive/dataset2'

Mounted at /content/drive


Move all the files to either Train/Test folders

In [63]:
import os
import random
import shutil

# Function to move images to train or test directories
def move_images(dataset_path, train_percent):
    # Create train and test directories
    train_dir = os.path.join(dataset_path, 'train')
    test_dir = os.path.join(dataset_path, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get all image files in the dataset directory
    image_files = [file for file in os.listdir(dataset_path) if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png')]

    # Calculate the number of images for the train and test sets
    num_train = int(len(image_files) * train_percent)
    num_test = len(image_files) - num_train

    # Randomly shuffle the image files
    random.shuffle(image_files)

    # Move images to train or test directories based on the specified percentage
    for i, filename in enumerate(image_files):
        if i < num_train:
            dest_dir = train_dir
        else:
            dest_dir = test_dir

        src_path = os.path.join(dataset_path, filename)
        dest_path = os.path.join(dest_dir, filename)
        shutil.move(src_path, dest_path)

# Specify the dataset path and train percentage
dataset_path = '/content/drive/My Drive/dataset2'
train_percent = 0.8  # Percentage of images for training set

# Move images to train or test directories
move_images(dataset_path, train_percent)


Create subfolders for Cloudy, Rain, Shine, Sunrise

In [64]:
train_path = '/content/drive/My Drive/dataset2/train'
test_path = '/content/drive/My Drive/dataset2/test'

def move_images_to_folders(dataset_path):
    # Iterate over each image file in the dataset directory
    for filename in os.listdir(dataset_path):
        # Extract label from the filename (alphabetic part)
        label = re.match(r'[a-zA-Z]+', filename).group().lower()

        # Create the label directory if it doesn't exist
        label_dir = os.path.join(dataset_path, label)
        os.makedirs(label_dir, exist_ok=True)

        # Move the image file to the label directory
        src_path = os.path.join(dataset_path, filename)
        dest_path = os.path.join(label_dir, filename)
        shutil.move(src_path, dest_path)

# Specify the dataset directory path
dataset_path = '/content/drive/My Drive/dataset2'

# Move images into folders based on the alphabetic part of the filename
move_images_to_folders(train_path)
move_images_to_folders(test_path)


Function for resetting folders

In [61]:
# import os
# import shutil

# # Function to move images to parent directory
# def move_images_to_parent(train_dir, test_dir):
#     # Iterate over each image file in train and test directories
#     for root_dir in [train_dir, test_dir]:
#         for root, _, files in os.walk(root_dir):
#             for file in files:
#                 src_path = os.path.join(root, file)
#                 dest_path = os.path.join(dataset_path, file)
#                 shutil.move(src_path, dest_path)
#         # Remove the train/test directory
#         shutil.rmtree(root_dir)

# # Specify the dataset path
# dataset_path = '/content/drive/My Drive/dataset2'

# # Path to train and test directories
# train_dir = os.path.join(dataset_path, 'train')
# test_dir = os.path.join(dataset_path, 'test')

# # Move images to parent directory
# move_images_to_parent(train_dir, test_dir)



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/dataset2/train'

Generate the images and batches of photos

In [8]:
import os
import shutil
from keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(rescale=1./255)

img_width, img_height = 224, 224

batch_size = 32

# Load and preprocess images from the train directory
train_generator = datagen.flow_from_directory(
    '/content/drive/My Drive/dataset2/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and preprocess images from the test directory
test_generator = datagen.flow_from_directory(
    '/content/drive/My Drive/dataset2/test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)



Found 900 images belonging to 4 classes.
Found 225 images belonging to 4 classes.


Create the CNN model and retrieve the results

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
28/28 [==============================] - 357s 13s/step - loss: 0.9742 - accuracy: 0.6728 - val_loss: 0.4450 - val_accuracy: 0.8839
Epoch 2/10
28/28 [==============================] - 117s 4s/step - loss: 0.5034 - accuracy: 0.8203 - val_loss: 0.3957 - val_accuracy: 0.8795
Epoch 3/10
28/28 [==============================] - 114s 4s/step - loss: 0.3727 - accuracy: 0.8664 - val_loss: 0.2422 - val_accuracy: 0.9196
Epoch 4/10
28/28 [==============================] - 113s 4s/step - loss: 0.2740 - accuracy: 0.8940 - val_loss: 0.3448 - val_accuracy: 0.9018
Epoch 5/10
28/28 [==============================] - 122s 4s/step - loss: 0.2264 - accuracy: 0.8963 - val_loss: 0.2860 - val_accuracy: 0.8884
Epoch 6/10
28/28 [==============================] - 112s 4s/step - loss: 0.2080 - accuracy: 0.9205 - val_loss: 0.3080 - val_accuracy: 0.9062
Epoch 7/10
28/28 [==============================] - 113s 4s/step - loss: 0.1457 - accuracy: 0.9435 - val_loss: 0.1622 - val_accuracy: 0.9330
Epoch 8/10
2

The accuracy for the model is 97.12 and the test accuraacy is 93.75, very good results.